In [1]:
# Import necessary libraries
from pathlib import Path
import os 
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
file_path_train = os.path.join(os.path.abspath('..'), "abnormal_detection_data/train/unswnb15_train_normal.csv")
file_path_test_normal = os.path.join(os.path.abspath('..'), "abnormal_detection_data/test/unswnb15_test_normal_full.csv")
file_path_test_abnormal = os.path.join(os.path.abspath('..'), "abnormal_detection_data/test/unswnb15_test_abnormal.csv")
df_train = pd.read_csv(file_path_train, index_col = 0)
df_test_normal = pd.read_csv(file_path_test_normal, index_col = 0)
df_test_abnormal = pd.read_csv(file_path_test_abnormal, index_col = 0)

In [3]:
df_test_normal = df_test_normal[:20000]
df_test = pd.concat([df_test_normal, df_test_abnormal])
df_test.columns = df_test_abnormal.columns

In [4]:
df_test_abnormal.shape, df_test_normal.shape

((45332, 39), (20000, 39))

In [7]:
# Define the score function for abnormal detection
def anomalyScores(originalDF, reducedDF):
  loss = np.sum((np.array(originalDF)-np.array(reducedDF))**2, axis=1)
  loss = pd.Series(data=loss,index=originalDF.index)
  loss = (loss-np.min(loss))/(np.max(loss)-np.min(loss))
  return loss

In [8]:
from sklearn.metrics import confusion_matrix

def results_analysis(df_gt_score, threshold, log=0):
  df_gt_pred = pd.DataFrame()
  df_gt_pred['ground_true'] = df_gt_score['ground_true']
  index = df_gt_score['anomalyScore'] > threshold
  df_gt_pred['prediction'] = index.astype(int)

  TN, FP, FN, TP = confusion_matrix(df_gt_pred['ground_true'], df_gt_pred['prediction']).ravel()
  precision_score = TP/(FP + TP)
  recall_score = TP/(FN + TP)
  accuracy_score = (TP + TN)/ (TP + FN + TN + FP)
  f1_score = 2*precision_score*recall_score/(precision_score + recall_score)
  fpr = FP / (FP+TN)
  fng = FN / (TP+FN)

  if log:
    print(f"Precision: {np.round(precision_score * 100.0,4)}%")
    print(f"Recall: {np.round(recall_score * 100.0,4)}%")
    print(f"Accuracy score: {np.round(accuracy_score * 100.0,4)}%")
    print(f"F1 score: {np.round(f1_score * 100.0,4)}%")
    print(f"False alarm: {np.round(fpr * 100.0,4)}%")
    print(f"False Negative: {np.round(fng * 100.0,4)}%")

  return precision_score, recall_score, accuracy_score, f1_score, fpr, fng

In [9]:
from sklearn.decomposition import PCA
scaler = StandardScaler()
def perform_pca(df_train, df_test):
  pca = PCA(0.99)
  pca.fit(scaler.transform(df_train))
  df_test_PCA = pca.transform(df_test)
  df_test_PCA_inverse = pca.inverse_transform(df_test_PCA)
  df_test_PCA = pd.DataFrame(df_test_PCA)
  df_test_PCA_inverse = pd.DataFrame(df_test_PCA_inverse)
  return df_test_PCA, df_test_PCA_inverse

# Change number of user in the cell below for experiments

In [ ]:
# df_test_raw = df_test.copy()
# df_normal_train = df_train.copy()
# num_users = 40
# fraction  = int(df_normal_train.shape[0] / num_users)

In [ ]:
# df_normal_train = df_normal_train.sort_values(by=['ct_srv_src'])
# df_normal_train = df_normal_train.reset_index(drop=True)


In [11]:
df_test_raw = df_test.copy()
df_normal_train = df_train.copy()
df_normal_train = df_normal_train.sort_values(by=['ct_srv_src'])
df_normal_train = df_normal_train.reset_index(drop=True)

In [13]:

user_experiments = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

for num_users in user_experiments:
  fraction  = int(df_normal_train.shape[0] / num_users)
  avg_acc = 0
  avg_pre = 0
  avg_rec = 0
  avg_f1 = 0
  avg_fpr = 0
  avg_fng = 0
  for i in range(num_users):
    df_train_stdPCA = df_normal_train[fraction*i:fraction*(i+1)].copy()
    df_train_client = df_train_stdPCA.copy()
    # Standardization over Testing
    scaler = StandardScaler()
    scaler.fit(df_train_client.to_numpy())
    df_test = pd.DataFrame(scaler.transform(df_test_raw.to_numpy()))
    df_test.columns = df_test_abnormal.columns
    _, df_test_PCA_inverse = perform_pca(df_train_client.to_numpy(), df_test.to_numpy())

    abnormal_score = anomalyScores(df_test, df_test_PCA_inverse)

    df_gt_score_PCA = pd.DataFrame(); df_gt_pred_PCA = pd.DataFrame()
    df_gt_score_PCA['ground_true'] = np.concatenate([np.zeros(len(df_test_normal)), np.ones(len(df_test_abnormal))])
    df_gt_score_PCA['anomalyScore'] = abnormal_score

    # choose the right threshold
    lst_p = np.arange(1e-1,9e-1,1e-1) # Among test, ratio of normal/abnormal = 0.75
    lst_rho = np.quantile(df_gt_score_PCA.anomalyScore, lst_p)
    optimal_p = 5e-1
    optimal_rho = lst_rho[abs(lst_p - optimal_p)<1e-8][0]
    
    precision_score, recall_score, accuracy_score, f1_score, fpr, fng = results_analysis(df_gt_score_PCA, threshold=optimal_rho, log=0)
    avg_acc += accuracy_score
    avg_pre += precision_score
    avg_rec += recall_score
    avg_f1 += f1_score
    avg_fpr += fpr
    avg_fng += fng
  print(f"--------------------Average results for {num_users} users--------------------")
  print(f"Precision: {np.round(avg_pre * 100.0/num_users,4)}%")
  print(f"Recall: {np.round(avg_rec * 100.0/num_users,4)}%")
  print(f"Accuracy score: {np.round(avg_acc * 100.0/num_users,4)}%")
  print(f"F1 score: {np.round(avg_f1 * 100.0/num_users,4)}%")
  print(f"False alarm: {np.round(avg_fpr * 100.0/num_users,4)}%")
  print(f"False Negative: {np.round(avg_fng * 100.0/num_users,4)}%")

--------------------Results for 10--------------------
Precision: 74.5887%
Recall: 53.7437%
Accuracy score: 55.199%
F1 score: 62.4733%
False alarm: 41.5025%
False Negative: 46.2563%
--------------------Results for 20--------------------
Precision: 71.8078%
Recall: 51.7289%
Accuracy score: 52.4137%
F1 score: 60.1366%
False alarm: 46.0342%
False Negative: 48.2711%
--------------------Results for 30--------------------
Precision: 73.7471%
Recall: 53.1358%
Accuracy score: 54.3571%
F1 score: 61.7673%
False alarm: 42.8747%
False Negative: 46.8642%
--------------------Results for 40--------------------
Precision: 72.1391%
Recall: 51.9732%
Accuracy score: 52.7472%
F1 score: 60.4178%
False alarm: 45.4982%
False Negative: 48.0268%
--------------------Results for 50--------------------
Precision: 71.4322%
Recall: 51.4644%
Accuracy score: 52.0405%
F1 score: 59.8262%
False alarm: 46.6537%
False Negative: 48.5356%
--------------------Results for 60--------------------
Precision: 70.9751%
Recall: 51.